In [1]:
"""
from google.colab import drive
drive.mount('/content/gdrive')
"""

"\nfrom google.colab import drive\ndrive.mount('/content/gdrive')\n"

In [2]:
import numpy as np
np.random.seed(1337)

import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [3]:
IMAGE_SIZE = [224, 224]

Now just remember the architecture in mind and start adding the layers into the network. our Kernel_size is (3,3) and the pool_size is (2,2) always.

In [4]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
model.add(Flatten(name='flatten'))
model.add(Dense(256, activation='relu', name='fc1'))
model.add(Dense(128, activation='relu', name='fc2'))
model.add(Dense(1, activation='sigmoid', name='output'))


In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

To use a pre-trained model just makes the middle layers non trainable and remove the last layer.

We are also going to do the same let’s makes the middle layers freeze.

In [6]:
for layer in vgg.layers:
  layer.trainable = False

We have already removed the output layer by include_top = False.

Let’s add our own output layer with only one node.

In [7]:
x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

Now we need to compile our model so that we can train it

In [8]:
model.compile(
  loss='categorical_crossentropy',
  #optimizer='adam',
  #optimizer='opt',
  optimizer='rmsprop',
  metrics=['accuracy']
)

Let’s generate training and validation data using the data generator

In [9]:
train_path="datasets/Buildings224/train"
test_path="datasets/Buildings224/test"

#train_path="/content/gdrive/My Drive/datasets/Buildings224/train"
#test_path="/content/gdrive/My Drive/datasets/Buildings224/test"

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   height_shift_range=0.3, 
                                   width_shift_range=0.3, 
                                   channel_shift_range=150.0, 
                                   brightness_range=(0.3, 0.9),
                                   validation_split=0.1) #validation split

test_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory(
    train_path,
    target_size = (224, 224),
    batch_size = 16,
    class_mode='categorical',
    subset='training') # set as training data

validation_set = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size = (224, 224),
    batch_size = 16,
    class_mode='categorical',
    subset='validation') # set as validation data

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                            shuffle=False)





Found 2145 images belonging to 5 classes.
Found 236 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


Train the model

In [10]:
r = model.fit(
  train_set,
  #steps_per_epoch = train_set.samples,
  validation_data = validation_set, 
  #validation_steps = validation_set.samples,
  epochs = 11,
  shuffle=False
)

Epoch 1/11
135/135 [==============================] - 609s 5s/step - loss: 1.1921e-07 - accuracy: 0.2277 - val_loss: 1.1921e-07 - val_accuracy: 0.2305
Epoch 2/11
135/135 [==============================] - 760s 6s/step - loss: 1.1921e-07 - accuracy: 0.2260 - val_loss: 1.1921e-07 - val_accuracy: 0.2254
Epoch 3/11
135/135 [==============================] - 791s 6s/step - loss: 1.1921e-07 - accuracy: 0.2338 - val_loss: 1.1921e-07 - val_accuracy: 0.2305
Epoch 4/11
135/135 [==============================] - 823s 6s/step - loss: 1.1921e-07 - accuracy: 0.2280 - val_loss: 1.1921e-07 - val_accuracy: 0.2305
Epoch 5/11
135/135 [==============================] - 767s 6s/step - loss: 1.1921e-07 - accuracy: 0.2269 - val_loss: 1.1921e-07 - val_accuracy: 0.2229
Epoch 6/11
135/135 [==============================] - 514s 4s/step - loss: 1.1921e-07 - accuracy: 0.2285 - val_loss: 1.1921e-07 - val_accuracy: 0.2280
Epoch 7/11
135/135 [==============================] - 366s 3s/step - loss: 1.1921e-07 - accura

Test the model

In [11]:
model.evaluate(test_set)

16/16 [==============================] - 33s 2s/step - loss: 1.1921e-07 - accuracy: 0.2168


[1.1920928955078125e-07, 0.2168000042438507]

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

print("predicting... ")

#Confution Matrix and Classification Report
Y_pred = model.predict(test_set, len(test_set), verbose = 1)


predicting... 
16/16 [==============================] - 27s 2s/step


In [13]:
#print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
#print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_set.labels, y_pred))
print('Classification Report')
target_names = ['Arch_Bridges', 'Castles', 'Churches', 'Towers', 'Triumphal_Arches']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

Confusion Matrix
[[50  0  0  0  0]
 [50  0  0  0  0]
 [50  0  0  0  0]
 [50  0  0  0  0]
 [50  0  0  0  0]]
Classification Report
                  precision    recall  f1-score   support

    Arch_Bridges       0.20      1.00      0.33        50
         Castles       0.00      0.00      0.00        50
        Churches       0.00      0.00      0.00        50
          Towers       0.00      0.00      0.00        50
Triumphal_Arches       0.00      0.00      0.00        50

        accuracy                           0.20       250
       macro avg       0.04      0.20      0.07       250
    weighted avg       0.04      0.20      0.07       250



/home/gecko/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gecko/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gecko/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
